In [26]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [27]:
# Créer une session Spark
spark = SparkSession.builder \
    .appName("Churn Prediction") \
    .getOrCreate()


In [28]:
# Charger les données
train_data = spark.read.csv("C:/Users/khoul/Dropbox/PC/Downloads/kafka-spark-streaming-integration-master/kafka-spark-streaming-integration-master/kafka/src/main/resources/churn-bigml-80.csv", header=True, inferSchema=True)
test_data = spark.read.csv("C:/Users/khoul/Dropbox/PC/Downloads/kafka-spark-streaming-integration-master/kafka-spark-streaming-integration-master/kafka/src/main/resources/churn-bigml-20.csv", header=True, inferSchema=True)


In [29]:
# Convertir la colonne Churn en string
train_data = train_data.withColumn("Churn", train_data["Churn"].cast("string"))
test_data = test_data.withColumn("Churn", test_data["Churn"].cast("string"))


In [30]:

# Convertir la colonne Churn en numérique
indexer = StringIndexer(inputCol="Churn", outputCol="label")
train_data = indexer.fit(train_data).transform(train_data)
test_data = indexer.fit(test_data).transform(test_data)


In [31]:

# Convertir les variables catégorielles en numériques
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(train_data) for column in ["State", "International plan", "Voice mail plan"]]
for indexer in indexers:
    train_data = indexer.transform(train_data)
    test_data = indexer.transform(test_data)

In [32]:
# Créer un vecteur de features
feature_columns = ["Account length", "Area code", "Number vmail messages", "Total day minutes", "Total day calls",
                   "Total day charge", "Total eve minutes", "Total eve calls", "Total eve charge", "Total night minutes",
                   "Total night calls", "Total night charge", "Total intl minutes", "Total intl calls", "Total intl charge",
                   "Customer service calls", "State_index", "International plan_index", "Voice mail plan_index"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
train_data = assembler.transform(train_data)
test_data = assembler.transform(test_data)

In [33]:

# Entraîner le modèle GBT avec maxBins augmenté
gbt = GBTClassifier(featuresCol="features", labelCol="label", maxBins=60)
gbt_model = gbt.fit(train_data)


In [34]:

# Faire des prédictions sur les données de test
predictions = gbt_model.transform(test_data)


In [12]:

# Évaluer les performances du modèle
evaluator = BinaryClassificationEvaluator(labelCol="label")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9003220463746787


In [15]:
spark.stop()

In [32]:
print("Accuracy:", accuracy)

Accuracy: 0.9003220463746787


In [33]:
predictions 

DataFrame[State: string, Account length: int, Area code: int, International plan: string, Voice mail plan: string, Number vmail messages: int, Total day minutes: double, Total day calls: int, Total day charge: double, Total eve minutes: double, Total eve calls: int, Total eve charge: double, Total night minutes: double, Total night calls: int, Total night charge: double, Total intl minutes: double, Total intl calls: int, Total intl charge: double, Customer service calls: int, Churn: string, label: double, State_index: double, International plan_index: double, Voice mail plan_index: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [35]:
# Save the model

gbt_model.write().overwrite().save("modelnew.pkl")  # Replace "model99" with your desired filename



In [36]:
from pyspark.ml.classification import GBTClassificationModel

# Load the model
loaded_model = GBTClassificationModel.load("modelnew.pkl")

# Suppose 'test_data' is your DataFrame containing the data to predict
predictions = loaded_model.transform(test_data)
# Show the predictions
predictions.show()

+-----+--------------+---------+------------------+---------------+---------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+----------------------+-----+-----+-----------+------------------------+---------------------+--------------------+--------------------+--------------------+----------+
|State|Account length|Area code|International plan|Voice mail plan|Number vmail messages|Total day minutes|Total day calls|Total day charge|Total eve minutes|Total eve calls|Total eve charge|Total night minutes|Total night calls|Total night charge|Total intl minutes|Total intl calls|Total intl charge|Customer service calls|Churn|label|State_index|International plan_index|Voice mail plan_index|            features|       rawPrediction|         probability|prediction|
+-----+--------------+---------+------------------+-------

In [37]:
# Show predictions as a table
predictions.select("features", "prediction").show()


+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[117.0,408.0,0.0,...|       0.0|
|[65.0,415.0,0.0,1...|       1.0|
|[161.0,415.0,0.0,...|       0.0|
|[111.0,415.0,0.0,...|       0.0|
|[49.0,510.0,0.0,1...|       0.0|
|[36.0,408.0,30.0,...|       0.0|
|[65.0,415.0,0.0,2...|       0.0|
|[119.0,415.0,0.0,...|       1.0|
|[10.0,408.0,0.0,1...|       0.0|
|[68.0,415.0,0.0,1...|       0.0|
|[74.0,510.0,33.0,...|       0.0|
|[85.0,415.0,0.0,2...|       0.0|
|[46.0,415.0,0.0,2...|       0.0|
|[128.0,510.0,29.0...|       0.0|
|[155.0,415.0,0.0,...|       0.0|
|[73.0,415.0,0.0,1...|       0.0|
|[77.0,415.0,0.0,2...|       0.0|
|[108.0,415.0,0.0,...|       0.0|
|[95.0,408.0,0.0,1...|       0.0|
|[36.0,510.0,29.0,...|       0.0|
+--------------------+----------+
only showing top 20 rows



In [73]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Chemin du fichier CSV
file_path = "C:/Users/khoul/Dropbox/PC/Downloads/kafka-spark-streaming-integration-master/kafka-spark-streaming-integration-master/spark/Master_dataset.csv"

# Lire le CSV sans spécifier le schéma
Newdata = spark.read.csv(file_path, header=False, inferSchema=True)

# Afficher les premières lignes
Newdata.show(5)

+-----+--------------+---------+------------------+---------------+--------------------+-----------------+---------------+----------------+-----------------+---------------+----------------+-------------------+-----------------+------------------+------------------+----------------+-----------------+--------------------+-------+----+----+----+----+----+-----+----+-----+-----+----+----+-----+----+----+----+----+----+----+-----+
|  _c0|           _c1|      _c2|               _c3|            _c4|                 _c5|              _c6|            _c7|             _c8|              _c9|           _c10|            _c11|               _c12|             _c13|              _c14|              _c15|            _c16|             _c17|                _c18|   _c19|_c20|_c21|_c22|_c23|_c24| _c25|_c26| _c27| _c28|_c29|_c30| _c31|_c32|_c33|_c34|_c35|_c36|_c37| _c38|
+-----+--------------+---------+------------------+---------------+--------------------+-----------------+---------------+----------------

In [71]:
predictions = loaded_model.transform(Newdata)


In [72]:
predictions.select("features", "prediction").show()


Py4JJavaError: An error occurred while calling o2293.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 531.0 failed 1 times, most recent failure: Lost task 0.0 in stage 531.0 (TID 553) (host.docker.internal executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda$3588/1696403424`: (struct<Account length_double_VectorAssembler_f319b2928594:double,Area code_double_VectorAssembler_f319b2928594:double,Number vmail messages_double_VectorAssembler_f319b2928594:double,Total day minutes:double,Total day calls_double_VectorAssembler_f319b2928594:double,Total day charge:double,Total eve minutes:double,Total eve calls_double_VectorAssembler_f319b2928594:double,Total eve charge:double,Total night minutes:double,Total night calls_double_VectorAssembler_f319b2928594:double,Total night charge:double,Total intl minutes:double,Total intl calls_double_VectorAssembler_f319b29285